In [1]:
# import packages
import os
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

## PromptTemplate

In [2]:
from langchain import PromptTemplate

template = "{name} is my favourite framework"

prompt = PromptTemplate(
	input_variables=["name"],
	template=template,
)

example = prompt.format(name="Lancgchain")

print(example)

Lancgchain is my favourite framework


## Memory
### ChatMessageHistory

La classe `ChatMessageHistory` est l'une des classes importantes des modules de langchain. Il s'agit d'un wrapper léger et pratique qui est utile pour enregistrer les messages humains et IA, puis les récupérer tous.

REMARQUE : Cette classe est généralement utilisée si vous essayez de gérer la mémoire en dehors d'une chaîne.

In [2]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message("Can you give me the solution for 10+10")
chat_history.add_ai_message("The Solution is 10+10=20")
chat_history.add_user_message("Thank You")

print(chat_history.messages)

[HumanMessage(content='Can you give me the solution for 10+10'), AIMessage(content='The Solution is 10+10=20'), HumanMessage(content='Thank You')]


### ConversationBufferMémoire

Cette classe est un wrapper sur la classe ChatMessageHistory et extrait les messages dans une variable.

In [3]:
from langchain.memory import ConversationBufferMemory

buff_memory = ConversationBufferMemory()

buff_memory.chat_memory.add_user_message("Can you give me the solution for 10+10")
buff_memory.chat_memory.add_ai_message("he Solution is 10+10=20")
buff_memory.chat_memory.add_user_message("ThankYou")

print(buff_memory.load_memory_variables)


<bound method ConversationBufferMemory.load_memory_variables of ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Can you give me the solution for 10+10'), AIMessage(content='he Solution is 10+10=20'), HumanMessage(content='ThankYou')]))>


## Memory and Chains

In [4]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import os

# defining chat model
chat_model = AzureChatOpenAI(
    openai_api_version="2023-09-01-preview",
    azure_endpoint=os.getenv('AZURE_API_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    azure_deployment=os.getenv('OPENAI_DEPLOYMENT_NAME'),
    model_name=os.getenv('OPENAI_MODEL_NAME'),
    model_version=os.getenv('OPENAI_API_VERSION')  # Add the desired version identifier
)

# defining chain
chain = ConversationChain(llm=chat_model,
    verbose=True,
    memory=ConversationBufferMemory()
    )
 
print(chain.predict(input="Can You tell me something about GeeksForGeeks."))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Can You tell me something about GeeksForGeeks.
AI:

> Finished chain.
Certainly! GeeksForGeeks is a website that provides resources for computer science and programming. It was founded in 2009 and is based in India. The website offers articles, tutorials, and coding challenges for a variety of programming languages and topics, including data structures, algorithms, web development, and more. It has a large community of developers who contribute to the content and forums on the site. Additionally, GeeksForGeeks offers courses and certifications for individuals looking to improve their programming skills.


In [5]:
# continuing with the memory
print(chain.predict(input="What are similar websites ?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Can You tell me something about GeeksForGeeks.
AI: Certainly! GeeksForGeeks is a website that provides resources for computer science and programming. It was founded in 2009 and is based in India. The website offers articles, tutorials, and coding challenges for a variety of programming languages and topics, including data structures, algorithms, web development, and more. It has a large community of developers who contribute to the content and forums on the site. Additionally, GeeksForGeeks offers courses and certifications for individuals looking to improve their programming skills.
Human: What are similar websites ?
AI:

> Finished chain.
There are a variety 

## Agent

Permet d'utiliser une collection d'outils et donner de la flexibilité.

In [7]:
# testing duckduckgo search
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.run("Obama's first name?")

"Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) 44th president of the United States (2009-17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005-08). He was the third African American to be elected to that body since the end of Reconstruction (1877). Barack Obama was the 44 th president of the United States and the first Black commander-in-chief. He served two terms, from 2009 until 2017. He served two terms, from 2009 until 2017. As the head of the government of the United States, the president is arguably the most powerful government official in the world. The president is elected to a four-year term via an electoral college system. Since the Twenty-second Amendment was adopted in 1951, the American presidency has been Barack and Michelle Obama's daughters, Malia and Sasha, grew up in the White House from 2009 to 2017. To most of the world, Barack and Michelle Obama are the former president

In [8]:
# testing wikipedia
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikipedia.run("HUNTER X HUNTER")

'Page: Hunter × Hunter\nSummary: Hunter × Hunter (stylized as HUNTER×HUNTER and pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 37 tankōbon volumes as of November 2022. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals. Gon departs on a journey to become a Hunter and eventually find his father. Along the way, Gon meets various other Hunters and encounters the paranormal.\nHunter × Hunter was adapted into a 62-episode anime television ser

In [13]:
# building an agent
from langchain.agents import initialize_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun

# Create instances of the tools
ddg_search_tool = DuckDuckGoSearchRun()
wikipedia_query_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Initialize the agent with the tools
agent = initialize_agent(
    tools=[ddg_search_tool, wikipedia_query_tool],
    llm=chat_model,  # Replace chat_model with your actual model
    agent="zero-shot-react-description",
    verbose=True
)

# Query
query = "Who is the president of the United States?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...
There might be a new president since the last time I checked.
Action: duckduckgo_search
Action Input: "current US president"
Observation: Joe Biden (born November 20, 1942, Scranton, Pennsylvania, U.S.) 46th president of the United States (2021- ) and 47th vice president of the United States (2009-17) in the Democratic administration of Pres. Barack Obama. He previously represented Delaware in the U.S. Senate (1973-2009). The president of the United States is the: U.S. head of state; Chief executive of the federal government; Commander-in-Chief of the armed forces; Current president. The 46th and current president of the United States is Joseph R. Biden, Jr. He was sworn into office on January 20, 2021. Former U.S. presidents. The United States has had 45 former ... A new poll of historians coming out on Presidents' Day weekend ranks Mr. Biden as the 14th-best president in American history, just ahead of Woodrow Wilson, Ronald Reagan and Ulysses 

In [15]:
query = "What is the history of the Eiffel Tower?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...
I think I need to use Wikipedia for this question.
Action: wikipedia
Action Input: "History of the Eiffel Tower"
Observation: Page: Eiffel Tower
Summary: The Eiffel Tower ( EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.
Locally nicknamed "La dame de fer" (French for "Iron Lady"), it was constructed as the centerpiece of the 1889 World's Fair, and to crown the centennial anniversary of the French Revolution. Although initially criticised by some of France's leading artists and intellectuals for its design, it has since become a global cultural icon of France and one of the most recognisable structures in the world.  The tower received 5,889,000 visitors in 2022. The Eiffel Tower is the most visited monument with an entrance fee in the world: 6.91 million people ascended it in

In [18]:
query = "Can langchain agent can use all the tools in a single call?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...
It's unlikely that a single call can utilize all the tools, but it's worth checking the documentation to confirm.
Action: DuckDuckGo search
Action Input: "langchain agent documentation"
Observation: DuckDuckGo search is not a valid tool, try one of [duckduckgo_search, wikipedia].
Thought:I made a mistake in my input, let me try again.
Action: DuckDuckGo search
Action Input: "can langchain agent use all tools in a single call"
Observation: DuckDuckGo search is not a valid tool, try one of [duckduckgo_search, wikipedia].
Thought:I need to find the documentation for Langchain Agent to see if it can use multiple tools in a single call.
Action: Wikipedia
Action Input: "Langchain Agent"
Observation: Wikipedia is not a valid tool, try one of [duckduckgo_search, wikipedia].
Thought:I need to clarify what "all the tools" means in the context of Langchain Agent.
Action: DuckDuckGo search
Action Input: "what tools can Langchain Agent use"
Observation: DuckD

Pour aller plus loin : https://blog.arcoptimizer.com/un-guide-complet-de-langchain-en-python-24